In [ ]:
from models import Models
from losses import *
from Line_of_horizont_fitting import Line_of_horizont_fitting
from Utils import Utils
import numpy as np

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = False  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

%load_ext autoreload
%autoreload 2 

%matplotlib inline

In [ ]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

In [ ]:
path_models='./models_saved/Final_new_dataset/'
img_w=160
img_h=160

model_class = Models()

model1=model_class.load_model(path_models+'unet160--mediumBN_mobilenetv2--dice_coef_loss')

In [ ]:
import matplotlib.pyplot as plt
import cv2
path_images = "./video_input/DS8/*"
path_2_save = "./video_input/DS8_result/"
for path_img, imageOUT, predict, img_inp_or, pred_inp_or in Utils.test_from_folder(path_images, model1, img_w, img_h, steps=300):
    name= path_img[path_img.rfind("/") + 1::]
    print(name)
    cv2.imwrite(path_2_save+"prediction-"+name, cv2.cvtColor(imageOUT, cv2.COLOR_RGB2BGR))
    cv2.imwrite(path_2_save+"mask_prediction-"+name, cv2.cvtColor(predict, cv2.COLOR_RGB2BGR))
    cv2.imwrite(path_2_save+"image_inpNN_or_size-"+name, cv2.cvtColor(img_inp_or, cv2.COLOR_RGB2BGR))
    cv2.imwrite(path_2_save+"mask_prediction_or_size-"+name, cv2.cvtColor(pred_inp_or, cv2.COLOR_RGB2BGR))

In [ ]:
filename = "video_input/prophr3.mp4"

Utils.test_from_video(filename, model1, img_w, img_h, 100)

In [ ]:
filename = "video_input/lakegarda-may-9-prop-8.avi"#prophr3.mp4"

Utils.test_speed_from_video(filename, model1, img_w, img_h, 100)

In [ ]:
from dataset import Dataset
path = 'Test/'
img_w = 1280
img_h = 720
n_labels = 2 #6
dataset = Dataset(path, img_w, img_h, n_labels)
x, y = dataset.createDataset(augmentation=False, prob1=False)

In [ ]:
model_class = Models()
path_models='./models_saved/final_models/'
model_path_name= ['unet160--mediumBN_mobilenetv2--dice_coef_loss']

for models_name in model_path_name:
    model1=model_class.load_model(path_models+models_name)
    recall_list, precision_list, specificity_list, accuracy_list, f1score_list = Utils.accuracy_on_images(x, y, model1, 160, 160, 1, False)
    print('*' * 30)
    print(models_name)
    print('N PARAMETERS =', model1.count_params())
    print("RECALL =", sum(recall_list) / float(len(recall_list)))
    print("PRECISION =", sum(precision_list) / float(len(precision_list)))
    print("SPECIFICITY =", sum(specificity_list) / float(len(specificity_list)))
    print("ACCURACY =", sum(accuracy_list) / float(len(accuracy_list)))
    print("F1SCORE =", sum(f1score_list) / float(len(f1score_list)))
    print('*' * 30)

In [ ]:
model_class = Models()
path_models='./models_saved/final_models/'
model_path_name= ['unet160--small--dice_coef_loss',
                  'unet160--smallBN--dice_coef_loss',
                  'unet160--medium--dice_coef_loss',
                  'unet160--mediumBN--dice_coef_loss']

for models_name in model_path_name:
    model1=model_class.load_model(path_models+models_name)
    recall_list, precision_list, specificity_list, accuracy_list, f1score_list = Utils.accuracy_on_line_of_horizont_area(x, y, model1, 160, 160, 1, False)
    print('*' * 30)
    print(models_name)
    print('N PARAMETERS =', model1.count_params())
    print("RECALL =", sum(recall_list) / float(len(recall_list)))
    print("PRECISION =", sum(precision_list) / float(len(precision_list)))
    print("SPECIFICITY =", sum(specificity_list) / float(len(specificity_list)))
    print("ACCURACY =", sum(accuracy_list) / float(len(accuracy_list)))
    print("F1SCORE =", sum(f1score_list) / float(len(f1score_list)))
    print('*' * 30)


In [ ]:
model_class = Models()
path_models='./models_saved/final_models/'
model_path_name= ['unet160--small--dice_coef_loss',
                  'unet160--smallBN--dice_coef_loss',
                  'unet160--medium--dice_coef_loss',
                  'unet160--mediumBN--dice_coef_loss']

for models_name in model_path_name:
    model1=model_class.load_model(path_models+models_name)
    avg_distance, max_distance = Utils.median_accuracy_line_of_horizont(x, y, model1, 160, 160, 1, False)
    print('*' * 30)
    print("-- ", models_name, " --")
    print("Mean Max error :", np.mean(max_distance))
    print("Median Max error :", np.median(max_distance))
    print("Average error :", np.mean(avg_distance))
    print("Max error :", np.max(max_distance))
    print('*' * 30)

In [ ]:
!cd models_saved/history/

In [ ]:
import pickle
import os
import matplotlib.pyplot as plt
history_path=['models_saved/history/unet160--history--largeBN--binary_crossentropy',
              'models_saved/history/unet160--history--largeBN--mae', 
              'models_saved/history/unet160--history--largeBN--mse',
              'models_saved/history/unet160--history--largeBN--dice_coef_loss', ]

historyes = []
for path in history_path:
    with (open(path+"", "rb")) as openfile:
        while True:
            try:
                historyes.append(pickle.load(openfile))
            except EOFError:
                break

In [ ]:
# list all data in history
print(historyes[0].keys())
# summarize history for loss
for history in historyes:
    plt.plot(history['val_dice_coef'])
plt.title('model accuracy')
plt.ylabel('Dice/F1 metric')
plt.xlabel('epoch')
plt.legend(['binary_crossentropy_loss',
            'mean_absolute_error_loss',
            'mean_squared_error_loss',
            'dice_coef_loss'], loc='lower right')
plt.axis([0,200,0.9,1])
plt.show()